In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc

In [4]:
"""
IMPORTANT NOTE:
pricing_data and all_data are global variables which should NOT be changed
They will ideally read from csv files; yet to be implented
"""

'\nIMPORTANT NOTE:\npricing_data and all_data are global variables which should NOT be changed\nThey will ideally read from csv files; yet to be implented\n'

In [5]:
from pandas.tseries.offsets import CustomBusinessDay
import pandas_market_calendars as mcal
valid_dates = mcal.get_calendar('NYSE').schedule(start_date = '2010-01-01', end_date = '2020-07-01').index

In [6]:
def zscore_factor_data(factor_data, date, factor):
    
    """
    Inputs: 
    factor_data - pandas dataframe having the raw factor data and same columns as daily_fundamentals dataframe
    date - date for which we want to compute factor z scores
    factor - factor name - like pb, pe whose zscores we want to get
    
    Returns:
    data[factor] - pandas series with index as tickers for that particular date and corressponding z score values
    """
    
    data = factor_data[factor_data['date'] == date]
    
    factor_arr = np.array(data[factor])
    mcap_arr = np.array(data['marketcap'])
    
    mcap_mean = np.nansum(factor_arr*mcap_arr)/np.nansum(mcap_arr)
    
    data[factor] = (data[factor] - mcap_mean)/data[factor].std()
       
    return data

In [7]:
def get_zscored_data(fundamentals, factors, dates):
    
    result = fundamentals
    for factor in factors:
        final = None
        for date in dates:
            raw_data = result[result['date'] == date]
            zscored_data = zscore_factor_data(raw_data, date, factor)
            if final is None:
                final = zscored_data
            else:
                final = pd.concat([final,zscored_data], axis = 0)
        result = final
    return result   

In [8]:
def validate_date(date):

    if(date in valid_dates):
        return date
    else:
        offset = CustomBusinessDay(n=1)
        return validate_date(date - offset)
    

In [9]:
def get_forward_returns(portfolio, period_end):
    
    """
    Inputs: 
    portfolio - python list containing stock tickers
    month_end - date of the month end 
    
    Returns:
    portfolio_return - total return on portfolio for the next month (scalar)
    
    """
    
    # Gets returns over next month from database for portfolio
    from pandas.tseries.offsets import BQuarterEnd
    offset = BQuarterEnd()
    period_end = offset.rollforward(period_end)
    next_period_end = period_end + offset
    period_end = validate_date(period_end)
    next_period_end = validate_date(next_period_end)

    portfolio_return = 0.0
    for stock in portfolio:
        try:
            ret = (pricing_data.loc[next_period_end, stock]/pricing_data.loc[period_end, stock]) - 1
            if(~np.isnan(ret)):
                portfolio_return = portfolio_return + ret 
        except:
            print("NO PRICING DATA FOR:", stock, period_end, next_period_end)       
    return portfolio_return 

In [10]:
def get_factor_returns(factors, period_end, lookback, universe):
    
    """
    Inputs: 
    factors - list of factors whose returns we want calculated
    month_end - date of the month end where new portfolio is selected
    lookback - number of months of historical data to use while calculating factor returns
    all_data - composite factor dataframe
    
    Returns:
    factor_returns: A matrix with rows as returns time series for each factor over the lookback period
    """

    dates = pd.date_range(end = period_end, periods = lookback+1, freq = 'Q')
    factor_returns = np.zeros([len(factors), lookback])
    dates = dates[:-1]
    i = 0 
    for factor in factors: 
        j = 0
        for date in dates:
            factor_data = (all_data[all_data['date'] == date]).set_index('ticker')
            factor_data = factor_data.loc[factor_data.index.intersection(universe)]
            long_portfolio = list(factor_data[factor].sort_values().tail(20).index)
            short_portfolio = list(factor_data[factor].sort_values().head(20).index)
            portfolio_ret = get_forward_returns(long_portfolio, date) - get_forward_returns(short_portfolio, date)
            factor_returns[i,j] = portfolio_ret
            j = j + 1
        i = i + 1
        
    
    return factor_returns

In [11]:
def get_historical_volatility_and_expected_returns(portfolio, period_end, lookback):
    
    """
    Inputs:
    portfolio - python list containing stock tickers
    month_end - date of the month end where new portfolio is selected
    lookback - number of months of historical data to use while calculating variance and expeceted returns
    
    Returns:
    asset_variances - numpy array of monthly asset variances for each asset
    expected_returns - numpy array of monthly asset expected returns for each asset
    """
    
    dates = pd.date_range(end = period_end, periods = lookback+1, freq = 'BQ')
    dates = dates[:-1]
    
    asset_variances = np.zeros(len(portfolio))
    expected_returns = np.zeros(len(portfolio))
    i = 0
    for stock in portfolio:
        
        returns_series = np.array(list(map(lambda x: get_forward_returns([stock], x), dates)))
        
        stock_variance = np.var(returns_series)/lookback# historical variance
        stock_mean_return = np.mean(returns_series)# historical expected return
        
        asset_variances[i] = stock_variance
        expected_returns[i] = stock_mean_return
        
        i = i + 1
    
    return asset_variances, expected_returns

In [12]:
def get_ret_risk_sharpe(weights, exposures, covariance_matrix, D_matrix, expected_returns):
    """
    Inputs:
    weights - numpy array of weights with length = number of assets
    exposures - matrix of factor values for stocks in portfolio of dimension (number of stocks x number of factors)
    covariance_matrix - estimated square covariance matrix of factor returns
    D_matrix - diagonal square matrix with diagonal elements as the historical variances of assets in our portfolio
    expected_returns - numpy array of estimated expected returns of stocks in portfolio using historical data

    Returns:
    ret - monthly return of portfolio with given weights 
    risk - monthly volatility of portfolio with given weights
    sharpe - sharpe ratio of portfolio with given weights; assumed that risk free rate = 0
    """
    systematic_risk_matrix = np.dot(exposures, np.dot(covariance_matrix, exposures.T))
    total_risk_matrix = systematic_risk_matrix + D_matrix  
    risk = np.sqrt((np.dot(weights, np.dot(total_risk_matrix, weights.T))))
    ret = np.dot(expected_returns.T,weights)
    sharpe = ret/risk
    
    return np.array([ret,risk,sharpe])

In [13]:
def neg_sharpe(weights, *args):
    
    """
    Inputs:
    weights - numpy array of weights with length = number of assets
    exposures - matrix of factor values for stocks in portfolio of dimension (number of stocks x number of factors)
    covariance_matrix - estimated square covariance matrix of factor returns
    D_matrix - diagonal square matrix with diagonal elements as the historical variances of assets in our portfolio
    expected_returns - numpy array of estimated expected returns of stocks in portfolio using historical data
        
    Returns negative sharpe ratio for use by optimisation function
    
    """
    ret = get_ret_risk_sharpe(weights, args[0], args[1], args[2], args[3])
    return ret[2]*(-1)

In [53]:
def check_sum(weights):
    
    """
    Checks if portfolio weights sum up to 1
    """
    
    return np.sum(weights) - 1

In [70]:
def get_optimised_weights(exposures, covariance_matrix, D_matrix, expected_returns, allocation):
    
    """
    weights - numpy array of weights with length = number of assets
    exposures - matrix of factor values for stocks in portfolio of dimension (number of stocks x number of factors)
    covariance_matrix - estimated square covariance matrix of factor returns
    D_matrix - diagonal square matrix with diagonal elements as the historical variances of assets in our portfolio
    expected_returns - numpy array of estimated expected returns of stocks in portfolio using historical data
    
    Returns:
    weights - numpy array of portfolio after maximising the sharpe ratio
    """
    
    def get_factor_weights(weights):
    
        factor_weights = np.dot(weights.T, exposures)
        factor_weights = factor_weights/np.sum(factor_weights)

        return factor_weights
    
    def growth_con(weights):
    
        factor_weights = get_factor_weights(weights)

        return allocation[1] - factor_weights[1]
    
    def em_con(weights):
    
        factor_weights = get_factor_weights(weights)

        return allocation[0] - factor_weights[0]
    
    def qual_con(weights):
    
        factor_weights = get_factor_weights(weights)

        return allocation[2] - factor_weights[2]
    
    
    asset_count = len(expected_returns)
    weights = np.ones(asset_count)/asset_count
    
    init_guess = weights 
    constraints = ({'type' : 'eq', 'fun': check_sum},
                   {'type' : 'ineq', 'fun':em_con}, 
                   {'type' : 'ineq', 'fun':growth_con}, 
                   {'type' : 'ineq', 'fun':qual_con})
    bounds = ((0,1),)*asset_count
    
    from scipy.optimize import minimize
    #Read documentation of scipy.optimize for greater detail regarding arguments of function
    opt_results = minimize(neg_sharpe, init_guess, args = (exposures, covariance_matrix, D_matrix, expected_returns),
                           method = "SLSQP", bounds = bounds, constraints = constraints)
    
    weights = opt_results.x
    
    return weights

In [71]:
def construction_analysis(period_end, factors, optimise, allocation):
    
    """
    Inputs:
    month_end - date of month end where new portfolio created
    all_data - composite monthly factor dataframe
    
    Returns:
    portfolio - list of stocks chosen for the upcoming month
    portfolio_weights - optimised weights for our portfolio of stocks
    ret - projected monthly return of portfolio using historical data
    risk - projected risk of portfolio using historical data
    sharpe - projected sharpe ratio of portfolio using projected return and risk
    """
    from pandas.tseries.offsets import BDay
    offset = BDay(n = 300)
    two_years_ago = validate_date(period_end - offset)
    universe = list((pricing_data.loc[two_years_ago, :].dropna()).index.unique())
    lookback_period = 4
    
    #Factor values should be normalized and free of missing values
    factor_data = (all_data[(all_data['date'] == period_end)]).set_index('ticker')[factors]
    factor_data = factor_data.loc[factor_data.index.intersection(universe)]
    
    #Now add all factors according to predetermined weights given by factor_weights
    factor_data['composite_score'] = factor_data.sum(axis = 1, numeric_only = True)
    
    #Sort stocks according to composite score
    total_score = factor_data[['composite_score']].sort_values(by = 'composite_score')
    
    #Use top 30 stocks for portfolio selection
    portfolio = list(total_score.tail(20).index)
    #Get monthly factor long-short portfolio returns for each of the last 12,18 months
    factor_returns_time_series = get_factor_returns(factors, period_end, lookback_period, universe)
    #factor returns should be no.of factors by lookback_period dimension matrix
    
    factor_returns = factor_returns_time_series.sum(axis =1)
    
    covariance_matrix = None
    if factor_returns_time_series.shape[0] == 1:
        covariance_matrix = np.var(factor_returns_time_series)
    else:
        covariance_matrix = np.cov(factor_returns_time_series.T, rowvar = False)

    
    #factor exposures matrix
    exposures = (factor_data.loc[portfolio,:])[factors].to_numpy()

    #get monthly volatility of all the assets in our portfolio based on past 12,18, months historical data
    asset_variance, expected_returns = get_historical_volatility_and_expected_returns(portfolio, period_end, lookback_period)
    expected_returns = np.dot(exposures, factor_returns)
    #contruct diagonal matrix of asset volatilities using np.diag
    D_matrix = np.diag(asset_variance)
    
    #perform mean-variance optimisation to obtain portfolio weights
    if optimise:
        portfolio_weights = get_optimised_weights(exposures,covariance_matrix, D_matrix, expected_returns, allocation)
    else:
        portfolio_weights = np.ones(len(portfolio), dtype = float)/len(portfolio)
    
    results = get_ret_risk_sharpe(portfolio_weights,exposures,covariance_matrix,D_matrix, expected_returns)
    ret = results[0]
    risk = results[1]
    sharpe = results[2]
        
    return [portfolio, portfolio_weights, ret, risk, sharpe]

In [72]:
def run(start_date, end_date, factors, optimise, allocation):
    
    """
    Inputs:
    start_date - starting date of analysis
    end_date - ending date of analysis
    NOTE - analysis is on a MONTHLTY BASIS ONLY. Choose start and end accordingly
    all_data - composite monthly factor dataframe
    
    Returns:
    monthly_analysis - python dictionary with keys as month ends, and values as a list of a single month's analysis
    """
    
    dates = pd.date_range(start_date, end_date, freq = 'Q')
    analysis = {}
    for period_end in dates:
        
        print(period_end)

        analysis[period_end] = construction_analysis(period_end, factors, optimise, allocation)
        
    return analysis        

In [19]:
import pickle
working_data = pickle.load(open('api_factor_data.p','rb'))

In [20]:
#working_data = raw_data[raw_data['is outlier'] == -1]
working_data['date'] = working_data['as_of_date']
working_data.drop(['as_of_date'], axis = 1, inplace = True)

In [21]:
start = pd.datetime(2015,1,1)
end = pd.datetime(2020,5,1)
dates = pd.date_range(start, end, freq = 'Q')

In [18]:
#working_data = get_zscored_data(working_data, ['eps'], dates)

In [22]:
all_data = working_data[['EarningsMomentum','Growth','Quality','ticker','date']]
print(all_data)

        EarningsMomentum  Growth  Quality ticker       date
0                 -5.747 -10.035   -4.404     CF 2017-03-31
1                 -2.017 -10.697   -1.764    FMC 2017-03-31
2                 -1.378  -6.374   -1.450    MOS 2017-03-31
3                 -6.957  -3.108   -3.229    NEM 2017-03-31
4                 -4.860 -10.280   -2.084   VEDL 2017-03-31
...                  ...     ...      ...    ...        ...
119619             2.210   0.898    0.152    CPK 2016-12-31
119620            -0.295   3.678    0.253   SPKE 2016-12-31
119621             2.235   9.552    0.218    UTL 2016-12-31
119622             2.208   3.735    1.309   MSEX 2016-12-31
119623             2.212   2.444   12.964  ARTNA 2016-12-31

[96431 rows x 5 columns]


In [23]:
pricing_data = pickle.load(open('pricing_data.p','rb'))

In [68]:
analysis_optimised = run(pd.datetime(2015,1,1), pd.datetime(2020,1,1), ['EarningsMomentum','Growth','Quality'], True)

2015-03-31 00:00:00
2015-06-30 00:00:00
2015-09-30 00:00:00
2015-12-31 00:00:00
2016-03-31 00:00:00
2016-06-30 00:00:00
2016-09-30 00:00:00
2016-12-31 00:00:00
2017-03-31 00:00:00
2017-06-30 00:00:00
2017-09-30 00:00:00
2017-12-31 00:00:00
2018-03-31 00:00:00
2018-06-30 00:00:00
2018-09-30 00:00:00
2018-12-31 00:00:00
2019-03-31 00:00:00
2019-06-30 00:00:00
2019-09-30 00:00:00
2019-12-31 00:00:00


In [62]:
analysis_unoptimised = run(pd.datetime(2015,1,1), pd.datetime(2020,1,1), ['EarningsMomentum','Growth','Quality'], False)

2015-03-31 00:00:00
2015-06-30 00:00:00
2015-09-30 00:00:00
2015-12-31 00:00:00
2016-03-31 00:00:00
2016-06-30 00:00:00
2016-09-30 00:00:00
2016-12-31 00:00:00
2017-03-31 00:00:00
2017-06-30 00:00:00
2017-09-30 00:00:00
2017-12-31 00:00:00
2018-03-31 00:00:00
2018-06-30 00:00:00
2018-09-30 00:00:00
2018-12-31 00:00:00
2019-03-31 00:00:00
2019-06-30 00:00:00
2019-09-30 00:00:00
2019-12-31 00:00:00


In [63]:
def backtest(analysis_optimised, analysis_unoptimised):
    
    for date in list(analysis_optimised.keys()):
        
        import pandas_market_calendars as mcal
        
        print(date)
        
        portfolio_optimised = analysis_optimised[date][0]
        portfolio_unoptimised = analysis_unoptimised[date][0]
        print("Portfolio: ",portfolio_optimised)
        
        next_quarter = validate_date(date + pd.tseries.offsets.BQuarterEnd(n=2))
        dates = mcal.get_calendar('NYSE').schedule(start_date = date, end_date = next_quarter).index
        
        def perform_analysis(portfolio, analysis):
            
            weights = np.array(analysis[date][1])
            portfolio_prices = list(map(lambda x: np.nansum(np.array(pricing_data.loc[x, portfolio])*weights),dates))
            portfolio_prices_ser = pd.Series(portfolio_prices, index = dates)
            init_cost = portfolio_prices[0]
            portfolio_returns = ((portfolio_prices_ser/portfolio_prices_ser.shift(1)) - 1).dropna()

            portfolio_vol = (portfolio_returns.std())*np.sqrt(63)
            portfolio_net_ret = (portfolio_prices_ser.iloc[-1]/portfolio_prices_ser.iloc[0]) - 1

            print("Portfolio Volatility: ",portfolio_vol*100)
            print("Portfolio Return: ",portfolio_net_ret*100)
            print("Sharp Ratio :", portfolio_net_ret/portfolio_vol)
            portfolio_prices = (portfolio_prices/portfolio_prices[0])*1000 # Notional 1000 USD Investment
            
            return portfolio_prices
        
        print("Optimised Portfolio Analysis: ")
        portfolio_prices_optimised = perform_analysis(portfolio_optimised, analysis_optimised)
        print()
        print("Unoptimised Portfolio Analysis: ")
        portfolio_prices_unoptimised = perform_analysis(portfolio_optimised, analysis_unoptimised)

        plot_df = pd.DataFrame({'Optimised Portfolio Price in USD': portfolio_prices_optimised,
                                'Unoptimised Portfolio Price in USD': portfolio_prices_unoptimised,
                                'Date':dates})
        
        fig_line_chart = px.line(plot_df,
                      x = 'Date',
                      y = ['Optimised Portfolio Price in USD', 'Unoptimised Portfolio Price in USD'],
                      title = 'Performance of Portfolios')
        
        fig_line_chart.update_xaxes(
            rangeslider_visible=True,
            rangeselector=dict(
            buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            )
        )
        fig_line_chart.show()
        
        weight_df = pd.DataFrame({'Stock': portfolio_optimised,
                                  'Equal Weights': analysis_unoptimised[date][1], 
                                  'Optimised Weights': analysis_optimised[date][1]})
        
        fig_bar = px.bar(weight_df,
                         x = 'Stock',
                         y = ['Equal Weights','Optimised Weights'],
                         title = 'Portfolio weights')
        fig_bar.update_layout(barmode = 'group')
        fig_bar.show()





In [69]:
backtest(analysis_optimised, analysis_unoptimised)

2017-12-31 00:00:00
Portfolio:  ['LIFE', 'CHRS', 'MYRG', 'AES', 'ALG', 'NRC', 'PG', 'AAWW', 'MNTA', 'ZBRA', 'NVEE', 'LEG', 'SYRS', 'BIOL', 'FORK', 'XNET', 'SQBG', 'INTU', 'NPO', 'CMCO']
Optimised Portfolio Analysis: 
Portfolio Volatility:  12.769490453230889
Portfolio Return:  -16.546918178444326
Sharp Ratio : -1.2958166372455124

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  8.592096049422635
Portfolio Return:  2.2084250235774983
Sharp Ratio : 0.2570298342656328


2016-09-30 00:00:00
Portfolio:  ['UGI', 'CHCI', 'QMCO', 'TMO', 'PATK', 'SJI', 'KSU', 'IKNX', 'CR', 'GDEN', 'MLM', 'PFE', 'MCFT', 'SJW', 'MNK', 'CJJD', 'EBAY', 'LCUT', 'AON', 'NRG']
Optimised Portfolio Analysis: 
Portfolio Volatility:  12.201309101798353
Portfolio Return:  40.35359110561081
Sharp Ratio : 3.3073165157059323

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  5.948779418440906
Portfolio Return:  6.282686966361517
Sharp Ratio : 1.0561304301997676


2017-03-31 00:00:00
Portfolio:  ['CTHR', 'ENDP', 'BMY', 'HBI', 'NEOG', 'MIND', 'XTNT', 'EFOI', 'ICE', 'IPG', 'ZBH', 'CWT', 'BBGI', 'HOFT', 'CXDC', 'PNM', 'IDXG', 'AXTA', 'GURE', 'BH']
Optimised Portfolio Analysis: 
Portfolio Volatility:  17.52663959614415
Portfolio Return:  -22.84175594847343
Sharp Ratio : -1.303259294126103

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  8.461540156233353
Portfolio Return:  -6.539408892988218
Sharp Ratio : -0.7728390780218468


2016-03-31 00:00:00
Portfolio:  ['LEG', 'ACU', 'OPNT', 'NUE', 'VRME', 'ATO', 'PFIN', 'BGFV', 'BWEN', 'RARE', 'HUM', 'HMHC', 'TECH', 'KAR', 'PCG', 'CNC', 'BLIN', 'A', 'VOD', 'GDEN']
Optimised Portfolio Analysis: 
Portfolio Volatility:  13.991620040693444
Portfolio Return:  9.055039636286665
Sharp Ratio : 0.647175924585634

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  14.856081612767843
Portfolio Return:  1.7862931176350516
Sharp Ratio : 0.12023985625522196


2015-12-31 00:00:00
Portfolio:  ['AY', 'CSGS', 'GURE', 'IMMU', 'IPI', 'MCO', 'ATSG', 'SENEA', 'GDEN', 'MANH', 'KFRC', 'TGC', 'MARA', 'NURO', 'EBAY', 'RGCO', 'AXAS', 'SGA', 'GLUU', 'SRL']
Optimised Portfolio Analysis: 
Portfolio Volatility:  13.27856234838318
Portfolio Return:  -0.7034633049751382
Sharp Ratio : -0.052977369576518434

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  11.501392231947857
Portfolio Return:  2.4654310358535714
Sharp Ratio : 0.2143593563399437


...

2018-12-31 00:00:00
Portfolio:  ['FLDM', 'CXO', 'RAMP', 'FWRD', 'MSCI', 'JKS', 'MEIP', 'INTU', 'EOG', 'JP', 'AAMC', 'EHTH', 'QD', 'WETF', 'SWKH', 'MORN', 'FCFS', 'EVR', 'TREE', 'ASPS']
Optimised Portfolio Analysis: 
Portfolio Volatility:  11.399846566716318
Portfolio Return:  47.04704157824953
Sharp Ratio : 4.1269890171689605

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  9.682903823133174
Portfolio Return:  40.33256306255348
Sharp Ratio : 4.165337568074983


2017-06-30 00:00:00
Portfolio:  ['DIN', 'RGEN', 'HQY', 'MASI', 'CWT', 'PWR', 'PNM', 'NTIP', 'CHUY', 'VISL', 'VNRX', 'PKI', 'EFOI', 'REV', 'BDSI', 'EXTR', 'HZNP', 'WTT', 'UONE', 'MEIP']
Optimised Portfolio Analysis: 
Portfolio Volatility:  5.275583777894685
Portfolio Return:  4.656749164909457
Sharp Ratio : 0.8826983630554371

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  5.778490669835563
Portfolio Return:  3.9936039166140525
Sharp Ratio : 0.6911154044880888


2019-12-31 00:00:00
Portfolio:  ['CMC', 'BV', 'MRVL', 'GOGO', 'BCRX', 'LTHM', 'FISV', 'EHTH', 'ROSE', 'HWKN', 'RBC', 'TRMB', 'LHX', 'JAGX', 'HRL', 'GM', 'ARCB', 'MD', 'MEIP', 'SRL']
Optimised Portfolio Analysis: 
Portfolio Volatility:  34.82600452206393
Portfolio Return:  -5.734500332656034
Sharp Ratio : -0.16466144799995516

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  24.80890543535165
Portfolio Return:  -8.694513376970992
Sharp Ratio : -0.35045937031069796


2016-06-30 00:00:00
Portfolio:  ['OPTT', 'BIG', 'CPAH', 'OMI', 'AMRN', 'AMSWA', 'MGNX', 'OFLX', 'GENC', 'WEN', 'GRMN', 'NWPX', 'GDEN', 'ISDR', 'ANF', 'LH', 'VCEL', 'QLYS', 'MRK', 'QTWO']
Optimised Portfolio Analysis: 
Portfolio Volatility:  25.229666035428473
Portfolio Return:  -16.724926977579724
Sharp Ratio : -0.6629071884698725

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  21.76290963969644
Portfolio Return:  6.936279578151283
Sharp Ratio : 0.3187202305660097


2016-12-31 00:00:00
Portfolio:  ['YTEN', 'PBH', 'GGG', 'ELMD', 'D', 'LPL', 'BAH', 'GILD', 'URG', 'AMZN', 'USAP', 'PRMW', 'ENR', 'VZ', 'PNM', 'NVCN', 'FISV', 'EGHT', 'SRL', 'TGNA']
Optimised Portfolio Analysis: 
Portfolio Volatility:  30.377944627032356
Portfolio Return:  -15.380027983251022
Sharp Ratio : -0.5062892888929961

Unoptimised Portfolio Analysis: 
Portfolio Volatility:  21.410416603188313
Portfolio Return:  -4.33136678926711
Sharp Ratio : -0.20230184538408788


In [65]:
import plotly.express as px
import plotly.graph_objects as go

In [39]:
type(pricing_data)

pandas.core.frame.DataFrame

In [60]:
print(analysis_optimised)

{Timestamp('2015-03-31 00:00:00', freq='Q-DEC'): [['SMG', 'SGRP', 'IRIX', 'SPSC', 'KAR', 'ALLE', 'CTG', 'SVT', 'USEG', 'PENN', 'QLYS', 'CORE', 'DIN', 'PETS', 'DDS', 'CDXS', 'EA', 'SHEN', 'SGMS', 'DVA'], array([0.04498105, 0.03960017, 0.0769008 , 0.003824  , 0.04852972,
       0.0515181 , 0.00310575, 0.26945864, 0.08385753, 0.0370571 ,
       0.00257215, 0.0565316 , 0.00320113, 0.0034846 , 0.08431728,
       0.00465585, 0.14141611, 0.02222788, 0.01508547, 0.00767506]), 3.7451585767959954, 7.219456929806591, 0.5187590442341385]}
